In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
def one_hot_encoder(df):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns.values[df.dtypes == 'object']]
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= True)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

## Initial browse through the dataset and initial modifications on the files

Application_train

In [3]:
df_application_train = pd.read_csv('application_train.csv')
df_application_train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
0  ...                 0                0                0                0   
1  ...                 0                0                0                0   
2  ...                 0                0                0                0   
3  ...                 0                0                0                0   
4  ...                 0                0                0                0   

  AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        NaN                       NaN   
4                        0.0                       0.0   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         NaN                        NaN   
4                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
0                        0.0                         1.0  
1                        0.0                         0.0  
2                        0.0                         0.0  
3                        NaN                         NaN  
4                        0.0                         0.0  

[5 rows x 122 columns]

In [4]:
df_application_train['CODE_GENDER'] = df_application_train['CODE_GENDER'].map(lambda x: x if x != 'XNA' else np.nan)
df_application_train['NAME_INCOME_TYPE'] = df_application_train['NAME_INCOME_TYPE'].map(lambda x: x if x != 'Maternity leave' else np.nan)
df_application_train['NAME_FAMILY_STATUS'] = df_application_train['NAME_FAMILY_STATUS'].map(lambda x: x if x != 'Unknown' else np.nan)

In [5]:
df_application_train, cat_cols_df_application_train = one_hot_encoder(df_application_train)

In [6]:
df_application_train['DAYS_EMPLOYED_PERC'] = df_application_train['DAYS_EMPLOYED'] / df_application_train['DAYS_BIRTH']
df_application_train['INCOME_CREDIT_PERC'] = df_application_train['AMT_INCOME_TOTAL'] / df_application_train['AMT_CREDIT']
df_application_train['PAYMENT_RATE'] = df_application_train['AMT_ANNUITY'] / df_application_train['AMT_CREDIT']
df_application_train['CREDIT_TO_GOODS_RATIO'] = df_application_train['AMT_CREDIT'] / df_application_train['AMT_GOODS_PRICE']
df_application_train['AMT_CREDIT_FRAC'] = df_application_train['AMT_GOODS_PRICE']/df_application_train['AMT_ANNUITY']

### Bureau_balance file

In [7]:
df_bureau_balance = pd.read_csv('bureau_balance.csv')
df_bureau_balance.head()

SK_ID_BUREAU  MONTHS_BALANCE STATUS
0       5715448               0      C
1       5715448              -1      C
2       5715448              -2      C
3       5715448              -3      C
4       5715448              -4      C

In [8]:
df_bureau_balance, cat_cols_df_bureau_balance = one_hot_encoder(df_bureau_balance)

### Bureau file

In [9]:
df_bureau = pd.read_csv('bureau.csv')
df_bureau.head()

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0      215354       5714462        Closed      currency 1         -497   
1      215354       5714463        Active      currency 1         -208   
2      215354       5714464        Active      currency 1         -203   
3      215354       5714465        Active      currency 1         -203   
4      215354       5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                     NaN                   0        225000.0   
2                     NaN                   0        464323.5   
3                     NaN                   0         90000.0   
4                 77674.5                   0       2700000.0   

   AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
0                  0.0                   NaN                     0.0   
1             171342.0                   NaN                     0.0   
2                  NaN                   NaN                     0.0   
3                  NaN                   NaN                     0.0   
4                  NaN                   NaN                     0.0   

       CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  
0  Consumer credit                -131          NaN  
1      Credit card                 -20          NaN  
2  Consumer credit                 -16          NaN  
3      Credit card                 -16          NaN  
4  Consumer credit                 -21          NaN

In [10]:
df_bureau['CREDIT_TO_ANNUITY_RATIO'] = df_bureau['AMT_CREDIT_SUM'] / df_bureau['AMT_ANNUITY']
df_bureau['BUREAU_CREDIT_DEBT_RATIO'] = df_bureau['AMT_CREDIT_SUM_DEBT'] / df_bureau['AMT_CREDIT_SUM']

In [11]:
df_bureau, cat_cols_df_bureau = one_hot_encoder(df_bureau)

### Installments_payments file

In [12]:
df_installments_payments = pd.read_csv('installments_payments.csv')
df_installments_payments.head()

SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  NUM_INSTALMENT_NUMBER  \
0     1054186      161674                     1.0                      6   
1     1330831      151639                     0.0                     34   
2     2085231      193053                     2.0                      1   
3     2452527      199697                     1.0                      3   
4     2714724      167756                     1.0                      2   

   DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  AMT_PAYMENT  
0          -1180.0             -1187.0        6948.360     6948.360  
1          -2156.0             -2156.0        1716.525     1716.525  
2            -63.0               -63.0       25425.000    25425.000  
3          -2418.0             -2426.0       24350.130    24350.130  
4          -1383.0             -1366.0        2165.040     2160.585

In [13]:
df_installments_payments, cat_cols_df_installments_payments = one_hot_encoder(df_installments_payments)
df_installments_payments['PAYMENT_PERC'] = df_installments_payments['AMT_PAYMENT'] / df_installments_payments['AMT_INSTALMENT']

Aggregations

In [14]:
aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'PAYMENT_PERC': ['mean','max','min'],
        'AMT_INSTALMENT': ['mean','min','max'],
        'AMT_PAYMENT': ['min','mean','sum','max'],
        'DAYS_ENTRY_PAYMENT': ['mean','min','max']
    }
for cat in cat_cols_df_installments_payments:
    aggregations[cat] = ['mean']
df_installments_agg = df_installments_payments.groupby('SK_ID_CURR').agg(aggregations)
df_installments_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in df_installments_agg.columns.tolist()])
df_installments_payments = df_installments_agg.copy()
del df_installments_agg
gc.collect()

0

### Credit_card_balance file

In [15]:
df_credit_card_balance = pd.read_csv('credit_card_balance.csv')
df_credit_card_balance.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0     2562384      378907              -6       56.970   
1     2582071      363914              -1    63975.555   
2     1740877      371185              -7    31815.225   
3     1389973      337855              -4   236572.110   
4     1891521      126868              -1   453919.455   

   AMT_CREDIT_LIMIT_ACTUAL  AMT_DRAWINGS_ATM_CURRENT  AMT_DRAWINGS_CURRENT  \
0                   135000                       0.0                 877.5   
1                    45000                    2250.0                2250.0   
2                   450000                       0.0                   0.0   
3                   225000                    2250.0                2250.0   
4                   450000                       0.0               11547.0   

   AMT_DRAWINGS_OTHER_CURRENT  AMT_DRAWINGS_POS_CURRENT  \
0                         0.0                     877.5   
1                         0.0                       0.0   
2                         0.0                       0.0   
3                         0.0                       0.0   
4                         0.0                   11547.0   

   AMT_INST_MIN_REGULARITY  ...  AMT_RECIVABLE  AMT_TOTAL_RECEIVABLE  \
0                 1700.325  ...          0.000                 0.000   
1                 2250.000  ...      64875.555             64875.555   
2                 2250.000  ...      31460.085             31460.085   
3                11795.760  ...     233048.970            233048.970   
4                22924.890  ...     453919.455            453919.455   

   CNT_DRAWINGS_ATM_CURRENT  CNT_DRAWINGS_CURRENT  CNT_DRAWINGS_OTHER_CURRENT  \
0                       0.0                     1                         0.0   
1                       1.0                     1                         0.0   
2                       0.0                     0                         0.0   
3                       1.0                     1                         0.0   
4                       0.0                     1                         0.0   

   CNT_DRAWINGS_POS_CURRENT  CNT_INSTALMENT_MATURE_CUM  NAME_CONTRACT_STATUS  \
0                       1.0                       35.0                Active   
1                       0.0                       69.0                Active   
2                       0.0                       30.0                Active   
3                       0.0                       10.0                Active   
4                       1.0                      101.0                Active   

   SK_DPD  SK_DPD_DEF  
0       0           0  
1       0           0  
2       0           0  
3       0           0  
4       0           0  

[5 rows x 23 columns]

In [16]:
df_credit_card_balance, cat_cols_df_credit_card_balance = one_hot_encoder(df_credit_card_balance)


Aggregations

In [17]:
df_credit_card_balance.drop(['SK_ID_PREV'], axis= 1, inplace = True)
df_credit_card_agg = df_credit_card_balance.groupby('SK_ID_CURR').agg(['mean'])
df_credit_card_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in df_credit_card_agg.columns.tolist()])
df_credit_card_balance = df_credit_card_agg.copy()
del df_credit_card_agg
gc.collect()

0

### POS_CASH_balance file

In [18]:
df_POS_CASH_balance = pd.read_csv('POS_CASH_balance.csv')
df_POS_CASH_balance.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
0     1803195      182943             -31            48.0   
1     1715348      367990             -33            36.0   
2     1784872      397406             -32            12.0   
3     1903291      269225             -35            48.0   
4     2341044      334279             -35            36.0   

   CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
0                   45.0               Active       0           0  
1                   35.0               Active       0           0  
2                    9.0               Active       0           0  
3                   42.0               Active       0           0  
4                   35.0               Active       0           0

In [19]:
df_POS_CASH_balance, cat_cols_df_POS_CASH_balance = one_hot_encoder(df_POS_CASH_balance)

Aggregations

In [20]:
aggregations = {
    'MONTHS_BALANCE': ['mean','min','max'],
    'SK_DPD': ['mean','min','max'],
    'SK_DPD_DEF': ['mean','min','max']
}
for cat in cat_cols_df_POS_CASH_balance:
    aggregations[cat] = ['mean']

df_POS_agg = df_POS_CASH_balance.groupby('SK_ID_CURR').agg(aggregations)
df_POS_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in df_POS_agg.columns.tolist()])
df_POS_CASH_balance = df_POS_agg.copy()
del df_POS_agg
gc.collect()

0

### Previous_application file

In [21]:
df_previous_application = pd.read_csv('previous_application.csv')
df_previous_application.head()

SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  \
0     2030495      271877     Consumer loans     1730.430          17145.0   
1     2802425      108129         Cash loans    25188.615         607500.0   
2     2523466      122040         Cash loans    15060.735         112500.0   
3     2819243      176158         Cash loans    47041.335         450000.0   
4     1784265      202054         Cash loans    31924.395         337500.0   

   AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  \
0     17145.0               0.0          17145.0                   SATURDAY   
1    679671.0               NaN         607500.0                   THURSDAY   
2    136444.5               NaN         112500.0                    TUESDAY   
3    470790.0               NaN         450000.0                     MONDAY   
4    404055.0               NaN         337500.0                   THURSDAY   

   HOUR_APPR_PROCESS_START  ... NAME_SELLER_INDUSTRY  CNT_PAYMENT  \
0                       15  ...         Connectivity         12.0   
1                       11  ...                  XNA         36.0   
2                       11  ...                  XNA         12.0   
3                        7  ...                  XNA         12.0   
4                        9  ...                  XNA         24.0   

   NAME_YIELD_GROUP       PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  \
0            middle  POS mobile with interest            365243.0   
1        low_action          Cash X-Sell: low            365243.0   
2              high         Cash X-Sell: high            365243.0   
3            middle       Cash X-Sell: middle            365243.0   
4              high         Cash Street: high                 NaN   

  DAYS_FIRST_DUE DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE DAYS_TERMINATION  \
0          -42.0                     300.0          -42.0            -37.0   
1         -134.0                     916.0       365243.0         365243.0   
2         -271.0                      59.0       365243.0         365243.0   
3         -482.0                    -152.0         -182.0           -177.0   
4            NaN                       NaN            NaN              NaN   

  NFLAG_INSURED_ON_APPROVAL  
0                       0.0  
1                       1.0  
2                       1.0  
3                       1.0  
4                       NaN  

[5 rows x 37 columns]

In [22]:
df_previous_application, cat_cols_df_previous_application = one_hot_encoder(df_previous_application)
df_previous_application['CREDIT_TO_ANNUITY_RATIO'] = df_previous_application['AMT_CREDIT'] / df_previous_application['AMT_ANNUITY']

Applications

In [23]:
num_aggregations = {
        'AMT_ANNUITY': ['mean','min','max'],
        'AMT_APPLICATION': ['mean','min','max'],
        'AMT_CREDIT': ['mean','min','max'],
        'AMT_DOWN_PAYMENT': ['mean','min','max'],
        'AMT_GOODS_PRICE': ['mean','min','max'],
        'HOUR_APPR_PROCESS_START': ['mean'],
        'RATE_DOWN_PAYMENT': ['mean','min','max'],
        'DAYS_DECISION': ['mean','min','max'],
        'CNT_PAYMENT': ['mean', 'sum','max'],
    }
cat_aggregations = {}
for cat in cat_cols_df_previous_application:
    cat_aggregations[cat] = ['mean']

df_previous_agg = df_previous_application.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
df_previous_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in df_previous_agg.columns.tolist()])

df_previous_application = df_previous_agg.copy()

del df_previous_agg
gc.collect()

0

## Aggregating into single file

Bureau balance into bureau

In [24]:
bb_aggregations = {'MONTHS_BALANCE': ['mean']}
for col in cat_cols_df_bureau_balance:
    bb_aggregations[col] = ['mean']
bb_agg = df_bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
df_bureau = df_bureau.join(bb_agg, how='left')#, on = 'SK_ID_BUREAU')
df_bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

num_aggregations = {
    'DAYS_CREDIT': ['mean','min','max'],
    'DAYS_CREDIT_ENDDATE': ['mean','min','max'],
    'DAYS_CREDIT_UPDATE': ['mean','min','max'],
    'CREDIT_DAY_OVERDUE': ['mean','min','max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean','min','max'],
    'AMT_CREDIT_SUM': ['mean','min','max'],
    'AMT_CREDIT_SUM_DEBT': ['mean','min','max'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean','min','max'],
    'AMT_CREDIT_SUM_LIMIT': ['mean','min','max'],
    'AMT_ANNUITY': ['mean','min','max'],
    'CNT_CREDIT_PROLONG': ['sum'],
}

cat_aggregations = {}
for cat in cat_cols_df_bureau: cat_aggregations[cat] = ['mean']
for cat in cat_cols_df_bureau_balance: cat_aggregations[cat + "_MEAN"] = ['mean']

bureau_agg = df_bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

active = df_bureau[df_bureau['CREDIT_ACTIVE_Active'] == 1]
active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

closed = df_bureau[df_bureau['CREDIT_ACTIVE_Closed'] == 1]
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
df_bureau = bureau_agg.copy()
del closed, closed_agg, bureau_agg, active, active_agg, df_bureau_balance, bb_agg
gc.collect()

0

Bureau into application_train

In [25]:
df_application_train = df_application_train.join(df_bureau, how='left', on='SK_ID_CURR')
del df_bureau
gc.collect()

0

Previous_application into application_train

In [26]:
df_application_train = df_application_train.join(df_previous_application, how='left', on='SK_ID_CURR')
del df_previous_application
gc.collect()

0

POS_cash_balance into application_train

In [27]:
df_application_train = df_application_train.join(df_POS_CASH_balance, how='left', on='SK_ID_CURR')
del df_POS_CASH_balance
gc.collect()

0

Installments_payments into application_train

In [28]:
df_application_train = df_application_train.join(df_installments_payments, how='left', on='SK_ID_CURR')
del df_installments_payments
gc.collect()

0

Credit_card_balance into application_train

In [29]:
df_application_train = df_application_train.join(df_credit_card_balance, how='left', on='SK_ID_CURR')
del df_credit_card_balance
gc.collect()

0

### Output into single file ready for analysis

In [30]:
df_application_train.to_csv('aggregated_train_data.csv')